# CS 109A/AC 209A/STAT 121A Data Science: Final Project
**Harvard University**<br>
**Fall 2016**<br>
**Instructors:** W. Pan, P. Protopapas, K. Rader<br>
**Members: ** Shawn Pan, Xinyuan (Amy) Wang, Ming-long Wu

## Remark - 055_ensemble.ipynb ##
- Do ensemble method
- Gradient Boosting R^2 0.65946833061


In [23]:
import time
import pandas as pd
import numpy as np
import scipy as sp
from scipy.signal import savgol_filter
from scipy.stats import mode
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib import colors
import matplotlib.dates as mdates
from datetime import datetime
from collections import Counter
from mpl_toolkits.basemap import Basemap
import statsmodels.api as sm
from sklearn import preprocessing
from collections import Counter
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
%matplotlib inline

### Load data

In [24]:
# Load listing data
listing = pd.read_csv('../../Data/data_after_predictor_selection.csv')
print(listing.columns.values)
listing.head()

['accommodates' 'bathrooms' 'bedrooms' 'review_scores_checkin'
 'review_scores_communication' 'latitude' 'longitude' 'property_type_0'
 'property_type_1' 'property_type_2' 'property_type_3' 'property_type_4'
 'property_type_5' 'room_type_0' 'room_type_1' 'room_type_2' 'bed_type_0'
 'bed_type_1' 'bed_type_2' 'bed_type_3' 'bed_type_4' 'beds'
 'review_scores_value' 'host_listing_count' 'review_scores_cleanliness'
 'review_scores_accuracy' 'minimum_nights' 'price']


,accommodates,bathrooms,bedrooms,review_scores_checkin,review_scores_communication,latitude,longitude,property_type_0,property_type_1,property_type_2,...,bed_type_2,bed_type_3,bed_type_4,beds,review_scores_value,host_listing_count,review_scores_cleanliness,review_scores_accuracy,minimum_nights,price
0,2,1.0,1.0,9.0,9.0,40.756852,-73.964754,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,9.0,1,7.0,9.0,3,160
1,10,1.0,3.0,9.0,8.0,40.830599,-73.941014,1.0,0.0,0.0,...,0.0,0.0,0.0,3.0,8.0,2,8.0,8.0,1,105
2,2,1.0,1.0,10.0,10.0,40.692189,-73.924120,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,10.0,4,10.0,10.0,3,58
3,2,1.0,1.0,10.0,10.0,40.734751,-74.002592,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,9.0,1,9.0,10.0,10,185
4,2,1.0,1.0,10.0,10.0,40.745282,-73.997836,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,10.0,1,10.0,10.0,1,195


## Feature engineering: location clustering to price by KNN ##

In [25]:
#Step to convert longitude and latitude to price cluster by KNN
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.neighbors import KNeighborsRegressor

class LocationPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = KNeighborsRegressor(n_neighbors=50, weights="uniform")
        super(LocationPreprocessor, self).__init__()
        
    def fit_transform(self, X, y):
        self.model.fit(X[["longitude", "latitude"]], y)
        return self.transform(X)
    
    def transform(self, X):
        loc = self.model.predict(X[["longitude", "latitude"]])
        Xnew = X.copy()
        Xnew["location"] = loc
        return Xnew

## Split data for training and testing data with transformation log(y)##

In [26]:
# Get x and y
x = listing
x = x.drop('price', axis=1)
y = listing['price']
print x.shape
print y.shape

(27392, 27)
(27392,)


In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, np.log(y), test_size=0.3, random_state=0)

### Ensemble - Gradient Boosting ###

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

n_est = 300
tuned_parameters = {
"n_estimators": [ n_est ],
"max_depth" : [ 4 ],
"learning_rate": [ 0.01 ],
"min_samples_split" : [ 1 ],
"loss" : [ 'ls', 'lad' ]
}
gbr = GradientBoostingRegressor()
clf = GridSearchCV(gbr, cv=3, param_grid=tuned_parameters)
preds = clf.fit(x_train, y_train)
best = clf.best_estimator_


In [29]:
print best

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.01, loss='ls',
             max_depth=4, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=1,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)


In [31]:
# fit with optimized parameter
start_time = time.time()
best.fit(x_train, y_train)
compute_time = time.time() - start_time

print "Train Score", best.score(x_train, y_train)
print "Test Score", best.score(x_test, y_test)
print "Compute Time", compute_time


Train Score 0.661300447664
Test Score 0.65946841902
Compute Time 7.26501512527


### Conclusion: Gradient Boosting performs similarly. ###